In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import load_model

In [2]:
dataset = pd.read_excel('Active Power.xlsx',sheet_name=0, header=0)
dataset

,测点编码,测点值,测点时间
0,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,0.000,2019-02-16 00:00:00
1,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.085,2019-02-16 00:15:00
2,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.061,2019-02-16 00:30:00
3,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.036,2019-02-16 00:45:00
4,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.011,2019-02-16 01:00:00
...,...,...,...
21787,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.043,2019-09-30 22:45:00
21788,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.089,2019-09-30 23:00:00
21789,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.068,2019-09-30 23:15:00
21790,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.097,2019-09-30 23:30:00


In [3]:
dataset.columns

Index(['测点编码', '测点值', '测点时间'], dtype='object')

In [4]:
dataset = pd.read_excel('Active Power.xlsx', sheet_name=0, header=0,
                        parse_dates={'datetime':[2]}, index_col=0)

In [5]:
dataset

,测点编码,测点值
datetime,,
2019-02-16 00:00:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,0.000
2019-02-16 00:15:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.085
2019-02-16 00:30:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.061
2019-02-16 00:45:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.036
2019-02-16 01:00:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.011
...,...,...
2019-09-30 22:45:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.043
2019-09-30 23:00:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.089
2019-09-30 23:15:00,xny_gt.nycg_s_xxxx_a_gfgf110kvcgxfhygz,-0.068


In [6]:
dataset.drop(columns=['测点编码'], inplace=True)

In [7]:
dataset

,测点值
datetime,
2019-02-16 00:00:00,0.000
2019-02-16 00:15:00,-0.085
2019-02-16 00:30:00,-0.061
2019-02-16 00:45:00,-0.036
2019-02-16 01:00:00,-0.011
...,...
2019-09-30 22:45:00,-0.043
2019-09-30 23:00:00,-0.089
2019-09-30 23:15:00,-0.068


In [8]:
dataset.columns

Index(['测点值'], dtype='object')

In [9]:
dataset.columns = [ 'value']

In [10]:
dataset

,value
datetime,
2019-02-16 00:00:00,0.000
2019-02-16 00:15:00,-0.085
2019-02-16 00:30:00,-0.061
2019-02-16 00:45:00,-0.036
2019-02-16 01:00:00,-0.011
...,...
2019-09-30 22:45:00,-0.043
2019-09-30 23:00:00,-0.089
2019-09-30 23:15:00,-0.068


In [11]:
dataset.isnull().sum()

value    0
dtype: int64

In [12]:
dataset.isna().sum()

value    0
dtype: int64

In [13]:
dataset.to_csv('ActivePower_Processed.csv')

In [14]:
dataset = pd.read_csv('ActivePower_Processed.csv', header=0,
                      infer_datetime_format=True, engine='c',
                      parse_dates=['datetime'], index_col=['datetime'])

In [15]:
dataset

,value
datetime,
2019-02-16 00:00:00,0.000
2019-02-16 00:15:00,-0.085
2019-02-16 00:30:00,-0.061
2019-02-16 00:45:00,-0.036
2019-02-16 01:00:00,-0.011
...,...
2019-09-30 22:45:00,-0.043
2019-09-30 23:00:00,-0.089
2019-09-30 23:15:00,-0.068


In [16]:
datasetX, evaluateX = dataset.iloc[:-35,:], dataset.iloc[-35:,:]

In [17]:
datasetX

,value
datetime,
2019-02-16 00:00:00,0.000
2019-02-16 00:15:00,-0.085
2019-02-16 00:30:00,-0.061
2019-02-16 00:45:00,-0.036
2019-02-16 01:00:00,-0.011
...,...
2019-09-30 14:00:00,33.669
2019-09-30 14:15:00,31.499
2019-09-30 14:30:00,29.780


In [18]:
evaluateX

,value
datetime,
2019-09-30 15:15:00,22.733
2019-09-30 15:30:00,20.452
2019-09-30 15:45:00,17.526
2019-09-30 16:00:00,14.660
2019-09-30 16:15:00,12.062
2019-09-30 16:30:00,9.724
2019-09-30 16:45:00,7.139
2019-09-30 17:00:00,4.733
2019-09-30 17:15:00,2.923


In [19]:
datasetX.shape

(21757, 1)

In [20]:
evaluateX.shape

(35, 1)

In [21]:
21757*0.7

15229.9

In [22]:
trainX, testX = datasetX.iloc[:15230,:], datasetX.iloc[15230:,:]

In [23]:
trainX.shape

(15230, 1)

In [24]:
testX.shape

(6527, 1)

In [25]:
def sliding_window(data, sw_width=7, n_out=7, in_start=0):
    '''
    该函数实现窗口宽度为7、滑动步长为1的滑动窗口截取序列数据
    '''
    X, y = [], []
    
    for _ in range(len(data)):
        in_end = in_start + sw_width
        out_end = in_end + n_out
        
        # 保证截取样本完整，最大元素索引不超过原序列索引，则截取数据；否则丢弃该样本
        if out_end < len(data):
            # 训练数据以滑动步长1截取
            train_seq = data[in_start:in_end, 0]
            train_seq = train_seq.reshape((len(train_seq), 1))
            X.append(train_seq)
            y.append(data[in_end:out_end, 0])
        in_start += 1
        
    return np.array(X), np.array(y)

In [55]:
def lstm_model(train, test, sw_width, in_start=0, verbose_set=1, epochs_num=5, batch_size_set=4):
    '''
    该函数定义 LSTM 模型
    单变量
    '''
    train_x, train_y = sliding_window(train, sw_width, in_start=0)
    test_x, test_y = sliding_window(test, sw_width, in_start=0)
    
    print('train_x.shape:{}, train_y.shape:{}'.format(train_x.shape, train_y.shape))
    print('test_x.shape:{}, test_y.shape:{}'.format(test_x.shape, test_y.shape))
    
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    
    model = Sequential()
    model.add(LSTM(200, activation='relu',
                   input_shape=(n_timesteps, n_features)))
    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_outputs))
    
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
    print(model.summary())
    
    model.save('lstm_active_power.h5')
    
    model.fit(train_x, train_y,
              validation_data = (test_x, test_y),
              epochs=epochs_num, batch_size=batch_size_set, verbose=verbose_set)
    
    return model

In [ ]:
sw_width = 7
in_start = 0

epochs_num = 30
batch_size_set = 128
verbose_set = 1


model = lstm_model(trainX.values, testX.values, sw_width, in_start, verbose_set, epochs_num, batch_size_set)

train_x.shape:(15216, 7, 1), train_y.shape:(15216, 7)
test_x.shape:(6513, 7, 1), test_y.shape:(6513, 7)
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 200)               161600    
_________________________________________________________________
dense_14 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 903       
Total params: 188,231
Trainable params: 188,231
Non-trainable params: 0
_________________________________________________________________
None
Train on 15216 samples, validate on 6513 samples
Epoch 1/30
15216/15216 [==============================] - 3s 213us/sample - loss: 56.4063 - accuracy: 0.3445 - val_loss: 25.5915 - val_accuracy: 0.2816
Epoch 2/30
15216/15216 [==============================] - 2s 118us/

In [25]:
evaluateX

,value
datetime,
2019-09-30 15:15:00,22.733
2019-09-30 15:30:00,20.452
2019-09-30 15:45:00,17.526
2019-09-30 16:00:00,14.660
2019-09-30 16:15:00,12.062
2019-09-30 16:30:00,9.724
2019-09-30 16:45:00,7.139
2019-09-30 17:00:00,4.733
2019-09-30 17:15:00,2.923


In [26]:
evaluateX.values[-14:-7]

array([[-0.122],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.179],
       [-0.089],
       [-0.089]])

In [27]:
test1 = evaluateX.values[0:7]
test1 = test1.reshape((1, len(test1), 1))

In [31]:
model = load_model('lstm_active_power.h5')

In [32]:
model.predict(test1)

array([[-0.16130035, -0.35880083, -0.00095478, -0.3806095 , -0.4357444 ,
        -0.7066136 ,  0.21375138]], dtype=float32)

In [34]:
evaluateX.values[7:14]

array([[ 4.733],
       [ 2.923],
       [ 1.392],
       [ 0.357],
       [-0.089],
       [-0.089],
       [-0.089]])

In [35]:
for i in range(4):
    test = evaluateX.values[i*7:(i+1)*7]
    test = test.reshape((1, len(test), 1))
    print(model.predict(test))

[[-0.16130035 -0.35880083 -0.00095478 -0.3806095  -0.4357444  -0.7066136
   0.21375138]]
[[ 0.00971176 -0.01107877  0.00381662 -0.015691   -0.00548593 -0.00677311
   0.00513768]]
[[ 0.00207362 -0.00069692  0.00026874 -0.00121145  0.00316218 -0.00084939
  -0.0003988 ]]
[[ 0.00471631 -0.00096969  0.0006064  -0.00240468  0.00628805 -0.00102353
  -0.00083284]]


In [37]:
evaluateX[7:].values

array([[ 4.733],
       [ 2.923],
       [ 1.392],
       [ 0.357],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.022],
       [-0.084],
       [ 0.   ],
       [-0.122],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.179],
       [-0.089],
       [-0.089],
       [-0.089],
       [-0.13 ],
       [-0.043],
       [-0.089],
       [-0.068],
       [-0.097],
       [-0.089]])